In [45]:
from src.components.transform import Transform
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('test').getOrCreate()

t = Transform(spark)

In [46]:
fo = t.transform()
fo.cache()

DataFrame[customer_city: string, customer_geo_lat_mean: double, customer_geo_lng_mean: double, customer_id: string, customer_state: string, customer_unique_id: string, customer_zip_code_prefix: string, freight_value: float, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_id: string, order_item_id: string, order_purchase_timestamp: timestamp, order_status: string, payment_installments: int, payment_sequential: int, payment_type: string, payment_value: float, price: float, product_category_name: string, product_description_lenght: int, product_height_cm: int, product_id: string, product_length_cm: int, product_name_lenght: int, product_photos_qty: int, product_weight_g: int, product_width_cm: int, review_answer_timestamp: timestamp, review_comment_message: string, review_comment_title: string, review_creation_date: timestamp, review_id: string, review_score: float, seller_city

In [47]:
fo.show()

+--------------------+---------------------+---------------------+--------------------+--------------+--------------------+------------------------+-------------+-------------------+----------------------------+-----------------------------+-----------------------------+--------------------+-------------+------------------------+------------+--------------------+------------------+------------+-------------+------+---------------------+--------------------------+-----------------+--------------------+-----------------+-------------------+------------------+----------------+----------------+-----------------------+----------------------+--------------------+--------------------+--------------------+------------+--------------------+-------------------+-------------------+--------------------+------------+----------------------+-------------------+
|       customer_city|customer_geo_lat_mean|customer_geo_lng_mean|         customer_id|customer_state|  customer_unique_id|customer_zip_code_

In [48]:
from pyspark.sql.functions import col

for column in fo.columns:
    na_counts = fo.select(column).filter(col(column).isNull()).count()

    print(f"Column: {column} -- NA {na_counts} \n")

Column: customer_city -- NA 0 

Column: customer_geo_lat_mean -- NA 317 

Column: customer_geo_lng_mean -- NA 317 

Column: customer_id -- NA 0 

Column: customer_state -- NA 0 

Column: customer_unique_id -- NA 0 

Column: customer_zip_code_prefix -- NA 0 

Column: freight_value -- NA 0 

Column: order_approved_at -- NA 15 

Column: order_delivered_carrier_date -- NA 1254 

Column: order_delivered_customer_date -- NA 2588 

Column: order_estimated_delivery_date -- NA 0 

Column: order_id -- NA 0 

Column: order_item_id -- NA 0 

Column: order_purchase_timestamp -- NA 0 

Column: order_status -- NA 0 

Column: payment_installments -- NA 3 

Column: payment_sequential -- NA 3 

Column: payment_type -- NA 3 

Column: payment_value -- NA 3 

Column: price -- NA 0 

Column: product_category_name -- NA 1709 

Column: product_description_lenght -- NA 1709 

Column: product_height_cm -- NA 0 

Column: product_id -- NA 0 

Column: product_length_cm -- NA 0 

Column: product_name_lenght -- NA 1

order-customer : 0 unmatch

order-items: 775 orders which order_id in orders dataset don't have a match with items dataset.

### Problems with Geoolocation Dataset
There are 161 zipcodes which are present in customers and sellers dataset but not in geolocation dataset.

In [ ]:
from src.config.config import GEOLOCATION_DATASET
from pyspark.sql.functions import col

geo = spark.read.csv(GEOLOCATION_DATASET, header=True, inferSchema=True)

In [ ]:
customer_zcode_notin_geodataset=(fo
                                 .filter(col('customer_geolocation_state').isNull())
                                 .select('customer_zip_code_prefix')
                                 .distinct()
                                 .toPandas()
                                 .loc[:, 'customer_zip_code_prefix']
                                 .tolist())

seller_zcode_notin_geodataset=(fo
                                 .filter(col('seller_geo_lng_mean').isNull())
                                 .select('seller_zip_code_prefix')
                                 .distinct()
                                 .toPandas()
                                 .loc[:, 'seller_zip_code_prefix']
                                 .tolist())


zipcode_not_in_dataset = set(customer_zcode_notin_geodataset + seller_zcode_notin_geodataset)
print('Number of zipcode not present in geolocation dataset:', len(zipcode_not_in_dataset))

161

In [99]:
from src.components.load import Load

l = Load(spark, mode='local')

In [100]:
l.load(fo, 'x')

In [101]:
import json

with open('pg_connection/pg_connection.json', 'r') as file:
    data = json.load(file)

data

{'connection_properties': {'username': 'username', 'password': 'password'}}

In [102]:
data['url'] = 's'

In [103]:
data

{'connection_properties': {'username': 'username', 'password': 'password'},
 'url': 's'}

In [1]:
from src.components.extract import Extract
from src.components.transform import Transform
from src.components.load import Load
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('s').getOrCreate()

c:\Users\ernes\Documents\ML Projects\spark-ecommerce\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
e = Extract()


In [3]:
e.extract()

nonononono


In [108]:
t = Transform(spark)

In [109]:
d = t.transform()